# Prototype WK01
## Create a data set of pharmacies in Germany

### Set-Up

In [1]:
import sys
!{sys.executable} -m pip install geopandas
!{sys.executable} -m pip install shapely
!{sys.executable} -m pip install geopy

In [2]:
import requests
import json
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point, shape
import numpy as np
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

### Get geometry data (here: Open Data API from Wegweiser Kommune)

In [3]:
headers = {
    'accept': 'application/json',
}

params = {
    'bbox': '5.98865807458,47.3024876979,15.0169958839,54.983104153',
    'layer': 'COMMUNE',
}

response = requests.get(
    'https://www.wegweiser-kommune.de/data-api/rest/map/data/demografische-entwicklung%2Bgeburten%2Bgemeinden-und-staedte%2B2021%2Bkarte',
    params=params,
    headers=headers,
)

In [4]:
kommune_data = response.json()

# Convert to a geopandas dataframe
kommune_df = gpd.GeoDataFrame.from_features(kommune_data['regions']['features'])

# Create a spatial index
kommune_sindex = kommune_df.sindex

In [31]:
kommune_df.head()

,geometry,gkz,values,name,classIndices,friendlyUrl,type,selected,pharmacy_count
0,"MULTIPOLYGON (((11.75800 48.14860, 11.74770 48...",09184118,[12.2],Feldkirchen (M),[6],feldkirchen-m,GEMEINDE,True,1.0
1,"MULTIPOLYGON (((8.64860 50.76270, 8.61800 50.7...",06534010,[9.2],Gladenbach,[4],gladenbach,GEMEINDE,True,4.0
2,"MULTIPOLYGON (((9.19270 47.97930, 9.19030 47.9...",08437078,[10.9],Meßkirch,[5],messkirch,GEMEINDE,True,2.0
3,"MULTIPOLYGON (((8.00460 50.06550, 7.98850 50.0...",06439014,[8.2],Schlangenbad,[4],schlangenbad,GEMEINDE,True,1.0
4,"MULTIPOLYGON (((8.90720 50.54960, 8.82380 50.5...",06531016,[10.4],Reiskirchen,[5],reiskirchen,GEMEINDE,True,2.0


### Get Open Street Maps Data from Overpass API

In [34]:
# Overpass API URL
overpass_url = "http://www.overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
area["ISO3166-1"="DE"];
(
node["amenity"="pharmacy"](area);
way["amenity"="pharmacy"](area);
rel["amenity"="pharmacy"](area);
node["healthcare"="pharmacy"](area);
way["healthcare"="pharmacy"](area);
rel["healthcare"="pharmacy"](area);
);
out center;
"""

# Send the request
response = requests.get(overpass_url, params={'data': overpass_query}) 
data = response.json()

In [35]:
len(data['elements'])

16869

### Match data

In [7]:
# Define empty list to hold pharmacy data 
pharmacies = []

# For each pharmacy
for element in data['elements']:
    if element['type'] == 'node':
        lon = element['lon']
        lat = element['lat']
    elif 'center' in element:
        lon = element['center']['lon']
        lat = element['center']['lat']
        
    # Create a point
    point = Point(lon, lat)

    # Get possible matches from the spatial index 
    possible_matches_index = list(kommune_sindex.intersection(point.bounds))
    possible_matches = kommune_df.iloc[possible_matches_index]

    # Check if point is in any of the possible matches 
    precise_matches = possible_matches[possible_matches.intersects(point)]

    # If a match is found
    match = precise_matches.iloc[0]['gkz'] if not precise_matches.empty else None

    pharmacies.append({'lon': lon, 'lat': lat, 'gkz': match})

# Create DataFrame from list
df = pd.DataFrame(pharmacies)

In [8]:
# Count the pharmacies per kommune
pharmacy_counts = df.groupby('gkz').size().reset_index(name='pharmacy_count')

# Merge the counts with the kommune_df
kommune_df = kommune_df.merge(pharmacy_counts, how='left', on='gkz')

# Fill NaN values with 0
kommune_df['pharmacy_count'] = kommune_df['pharmacy_count'].fillna(0)

print(kommune_df)

                                               geometry       gkz  values  \
0     MULTIPOLYGON (((11.75800 48.14860, 11.74770 48...  09184118  [12.2]   
1     MULTIPOLYGON (((8.64860 50.76270, 8.61800 50.7...  06534010   [9.2]   
2     MULTIPOLYGON (((9.19270 47.97930, 9.19030 47.9...  08437078  [10.9]   
3     MULTIPOLYGON (((8.00460 50.06550, 7.98850 50.0...  06439014   [8.2]   
4     MULTIPOLYGON (((8.90720 50.54960, 8.82380 50.5...  06531016  [10.4]   
...                                                 ...       ...     ...   
3057  MULTIPOLYGON (((12.16350 50.95870, 12.09230 50...  15084952   [6.3]   
3058  MULTIPOLYGON (((11.08350 51.82660, 11.05820 51...  15085951   [4.8]   
3059  MULTIPOLYGON (((10.96270 51.46030, 10.97270 51...  15087951   [5.6]   
3060  MULTIPOLYGON (((11.75910 51.28370, 11.70120 51...  15088951   [4.6]   
3061  MULTIPOLYGON (((11.44610 52.00700, 11.45080 51...  15089951   [5.2]   

                         name classIndices               friendlyUrl  \
0  

In [9]:
kommune_df.to_csv('data/output/pharmacydata.csv', index=False, sep =";")

In [10]:
kommune_df.head()

,geometry,gkz,values,name,classIndices,friendlyUrl,type,selected,pharmacy_count
0,"MULTIPOLYGON (((11.75800 48.14860, 11.74770 48...",09184118,[12.2],Feldkirchen (M),[6],feldkirchen-m,GEMEINDE,True,1.0
1,"MULTIPOLYGON (((8.64860 50.76270, 8.61800 50.7...",06534010,[9.2],Gladenbach,[4],gladenbach,GEMEINDE,True,4.0
2,"MULTIPOLYGON (((9.19270 47.97930, 9.19030 47.9...",08437078,[10.9],Meßkirch,[5],messkirch,GEMEINDE,True,2.0
3,"MULTIPOLYGON (((8.00460 50.06550, 7.98850 50.0...",06439014,[8.2],Schlangenbad,[4],schlangenbad,GEMEINDE,True,1.0
4,"MULTIPOLYGON (((8.90720 50.54960, 8.82380 50.5...",06531016,[10.4],Reiskirchen,[5],reiskirchen,GEMEINDE,True,2.0


In [11]:
kommune_df['gkz']=kommune_df['gkz'].astype(str)

### Quality Assurance

In [12]:
# tabelle: gkz, name, count pharmacies

In [ ]:
# qc-datei: stichprobe an apotheken -> geompoints (google)

In [13]:
qc_data = pd.read_excel("data/qc_apothekenregisterstichprobe.xlsx", sheet_name = "Values", dtype = {'gkz': str})

In [14]:
qc_data.head()

,gkz,GKZ Ordnen,ARS,Name,Ausgabe im Wegweiser Kommune,typ_bezeichnung,auswahl,plz,anzahl apotheken,notiz
0,08236076,8236076,82365002076,Königsbach-Stein,Königsbach-Stein (im Enzkreis),Gemeinde mit 5.000 oder mehr Einwohnern,0.999521,75203,3.0,NaN
1,08415059,8415059,84150059059,Pfullingen,Pfullingen (im Landkreis Reutlingen),Gemeinde mit 5.000 oder mehr Einwohnern,0.999435,72793,6.0,NaN
2,14628110,14628110,146280110110,Freital,Freital (im Landkreis Sächsische Schweiz-Oster...,Gemeinde mit 5.000 oder mehr Einwohnern,0.999347,01705,8.0,pesterwitz zu freital?
3,13074026,13074026,130740026026,Grevesmühlen,Grevesmühlen (im Landkreis Nordwestmecklenburg),Gemeinde mit 5.000 oder mehr Einwohnern,0.999173,23936,4.0,NaN
4,05566068,5566068,55660068068,Ochtrup,Ochtrup (im Landkreis Steinfurt),Gemeinde mit 5.000 oder mehr Einwohnern,0.999123,48607,5.0,NaN


In [15]:
qc_df = qc_data.merge(kommune_df, on = "gkz")[['gkz', 'name', 'anzahl apotheken', 'pharmacy_count']]

In [16]:
qc_df['true'] = qc_df['pharmacy_count'] == qc_df['anzahl apotheken']
qc_df['percentage'] = qc_df['pharmacy_count']/qc_df['anzahl apotheken']

In [17]:
qc_df[qc_df['percentage'] > 1]

,gkz,name,anzahl apotheken,pharmacy_count,true,percentage
8,10044111,Dillingen / Saar,8.0,11.0,False,1.375000
12,05970024,Kreuztal,3.0,6.0,False,2.000000
27,15091241,Oranienbaum-Wörlitz,2.0,3.0,False,1.500000
35,07235907,Trier-Land,1.0,2.0,False,2.000000
39,06534021,Wetter (Hessen),2.0,3.0,False,1.500000
49,15083953,Obere Aller,3.0,5.0,False,1.666667
52,14521260,Grünhain-Beierfeld,1.0,2.0,False,2.000000
58,13075136,Ueckermünde,3.0,4.0,False,1.333333
62,05158032,Velbert,19.0,20.0,False,1.052632
63,03458014,Wildeshausen,4.0,5.0,False,1.250000


In [18]:
qc_df[qc_df['percentage'] < 1]

,gkz,name,anzahl apotheken,pharmacy_count,true,percentage
1,08415059,Pfullingen,6.0,5.0,False,0.833333
7,03454018,Haren (Ems),9.0,6.0,False,0.666667
10,03459403,Fürstenau,3.0,2.0,False,0.666667
15,06533004,Dornburg,2.0,1.0,False,0.500000
16,09189157,Trostberg,2.0,1.0,False,0.500000
18,05154012,Geldern,9.0,8.0,False,0.888889
19,08125102,Weinsberg,3.0,2.0,False,0.666667
22,08337123,Wutöschingen,1.0,0.0,False,0.000000
26,03453002,Bösel,2.0,1.0,False,0.500000
30,03356009,Schwanewede,3.0,2.0,False,0.666667


In [19]:
qc_df[qc_df['percentage'] == 1]

,gkz,name,anzahl apotheken,pharmacy_count,true,percentage
0,08236076,Königsbach-Stein,3.0,3.0,True,1.0
2,14628110,Freital,8.0,8.0,True,1.0
3,13074026,Grevesmühlen,4.0,4.0,True,1.0
4,05566068,Ochtrup,5.0,5.0,True,1.0
5,06633003,Baunatal,7.0,7.0,True,1.0
6,05570012,Beelen,1.0,1.0,True,1.0
9,03356007,Osterholz-Scharmbeck,6.0,6.0,True,1.0
11,05962048,Neuenrade,1.0,1.0,True,1.0
14,14521640,Thum,1.0,1.0,True,1.0
17,09189145,Siegsdorf,2.0,2.0,True,1.0


In [20]:
qc_df['percentage'].describe()

count    84.000000
mean      0.935532
std       0.456444
min       0.000000
25%       0.729167
50%       1.000000
75%       1.000000
max       2.000000
Name: percentage, dtype: float64

In [21]:
sum(qc_df['true'])/len(qc_df)

0.4659090909090909

In [22]:
len(qc_df['true'])

88

In [23]:
qc_df.to_csv('data/output/qcresultsdata.csv', index=False, sep =";")

Questions:
- Issue of geographical mapping and matching?
    - Test 1: Check OSM data for wrong matches directly
- Issue of data quality?
    - Data enrichment
    - Use of Google Maps API

### Data enrichtment: Handelsregisterdaten

https://offeneregister.de/daten/

In [24]:
import sqlite3

def get_table_structure(cursor, table_name):
    cursor.execute(f"PRAGMA table_info({table_name})")
    return cursor.fetchall()

conn = sqlite3.connect('data/handelsregister.db')

c = conn.cursor()

c.execute("SELECT name FROM sqlite_master WHERE type = 'table';")

tables = c.fetchall()

for table in tables:
    print(f"Table: {table[0]}")
    structure = get_table_structure(c, table[0])
    for column in structure:
        print(column)

Table: name
(0, 'id', 'INTEGER', 1, None, 1)
(1, 'company_name', 'TEXT', 0, None, 0)
(2, 'company_id', 'TEXT', 0, None, 0)
Table: registrations
(0, 'id', 'INTEGER', 1, None, 1)
(1, 'confidence', 'TEXT', 0, None, 0)
(2, 'data_type', 'TEXT', 0, None, 0)
(3, 'publication_date', 'TEXT', 0, None, 0)
(4, 'retrieved_at', 'TEXT', 0, None, 0)
(5, 'source_url', 'TEXT', 0, None, 0)
(6, 'start_date', 'TEXT', 0, None, 0)
(7, 'start_date_type', 'TEXT', 0, None, 0)
(8, 'subsequent_registration_start_date', 'TEXT', 0, None, 0)
(9, 'company_id', 'TEXT', 0, None, 0)
(10, 'previous_company_number', 'TEXT', 0, None, 0)
(11, 'previous_jurisdiction_code', 'TEXT', 0, None, 0)
(12, 'previous_entity_type', 'TEXT', 0, None, 0)
(13, 'subsequent_company_number', 'TEXT', 0, None, 0)
(14, 'subsequent_jurisdiction_code', 'TEXT', 0, None, 0)
(15, 'subsequent_entity_type', 'TEXT', 0, None, 0)
(16, 'sample_date', 'TEXT', 0, None, 0)
(17, 'alternate_company_number', 'TEXT', 0, None, 0)
(18, 'alternate_jurisdiction_code'

In [25]:
# select entries from table: company with columns id, name and registered_address: Germany has approx. 17,000 pharmacies
registry_df = pd.read_sql_query('SELECT * FROM company WHERE name LIKE "%Apotheke%"', conn)
print(len(registry_df))
registry_df.head()

33838


,id,company_number,current_status,jurisdiction_code,name,registered_address,retrieved_at,register_flag_AD,register_flag_CD,register_flag_DK,...,native_company_number,registered_office,registrar,register_art,register_nummer,former_registrar,register_flag_,register_flag_Note:,_registerNummerSuffix,register_flag_Status information
0,576,R1101_HRA17543,removed,de,Linden-Apotheke Georg Reibel e.K.,None,2018-07-25T13:02:09Z,0,1,1,...,Düsseldorf HRA 17543,Hilden,Düsseldorf,HRA,17543,None,None,None,None,None
1,786,F1103R_HRA38448,currently registered,de,Apotheke in Nikolassee e.K.,None,2018-11-09T18:29:04Z,1,1,1,...,Berlin (Charlottenburg) HRA 38448,Berlin,Berlin (Charlottenburg),HRA,38448,None,None,None,None,None
2,1007,M1103_HRB3094,currently registered,de,CIDA Computerleistungen für Apotheken GmbH,None,2018-06-22T05:25:39Z,1,1,1,...,Darmstadt HRB 3094,Darmstadt,Darmstadt,HRB,3094,None,None,None,None,None
3,1453,M1201_HRA49032,currently registered,de,Westend Apotheke Homira Alavi e.K.,"Brentano Straße 29, 60325 Frankfurt am Main.",2018-07-25T13:36:39Z,1,1,1,...,Frankfurt am Main HRA 49032,Frankfurt am Main,Frankfurt am Main,HRA,49032,None,None,None,None,None
4,1971,U1206_HRA658,currently registered,de,"Löwen-Apotheke Apothekerin Constanze Köhler, e.K.","Erich-Kästner-Straße 32, 09669 Frankenberg/Sa.",2018-11-09T10:12:34Z,1,1,1,...,Chemnitz HRA 658,Frankenberg/Sa.,Chemnitz,HRA,658,None,None,None,None,None


In [26]:
# Drop duplicates
registry_df = registry_df.drop_duplicates(subset = ['register_nummer', 'register_art'], keep = 'last').reset_index(drop = True)
len(registry_df)

21819

In [27]:
# filter actice pharmacies - are hrb entries relevant?
registry_df = registry_df.loc[(registry_df['current_status'] == "currently registered") & (registry_df['register_art'] == "HRA")]
print(len(registry_df))
registry_df.head()

11384


,id,company_number,current_status,jurisdiction_code,name,registered_address,retrieved_at,register_flag_AD,register_flag_CD,register_flag_DK,...,native_company_number,registered_office,registrar,register_art,register_nummer,former_registrar,register_flag_,register_flag_Note:,_registerNummerSuffix,register_flag_Status information
1,786,F1103R_HRA38448,currently registered,de,Apotheke in Nikolassee e.K.,None,2018-11-09T18:29:04Z,1,1,1,...,Berlin (Charlottenburg) HRA 38448,Berlin,Berlin (Charlottenburg),HRA,38448,None,None,None,None,None
3,1453,M1201_HRA49032,currently registered,de,Westend Apotheke Homira Alavi e.K.,"Brentano Straße 29, 60325 Frankfurt am Main.",2018-07-25T13:36:39Z,1,1,1,...,Frankfurt am Main HRA 49032,Frankfurt am Main,Frankfurt am Main,HRA,49032,None,None,None,None,None
5,2541,R1402_HRA5334,currently registered,de,"Bären-Apotheke, Inh.: Olaf Orthen e.K.",None,2018-07-25T14:18:03Z,1,1,1,...,Krefeld HRA 5334,Kempen,Krefeld,HRA,5334,None,None,None,None,None
6,2817,B8535_HRA440044,currently registered,de,"Apotheke Billigheim Wolfgang Pretzsch-Eckhard,...",None,2018-11-09T09:59:11Z,1,1,1,...,Mannheim HRA 440044,Billigheim,Mannheim,HRA,440044,None,None,None,None,None
7,2989,B8535_HRA340724,currently registered,de,Stadt-Apotheke Volker Blum e.K.,None,2018-11-09T09:58:28Z,1,1,1,...,Mannheim HRA 340724,Sinsheim,Mannheim,HRA,340724,None,None,None,None,None


In [28]:
registry_df['searchable_address'] = registry_df['registered_address'] + ' ' + registry_df['registered_office']

In [29]:
registry_df.head()

,id,company_number,current_status,jurisdiction_code,name,registered_address,retrieved_at,register_flag_AD,register_flag_CD,register_flag_DK,...,registered_office,registrar,register_art,register_nummer,former_registrar,register_flag_,register_flag_Note:,_registerNummerSuffix,register_flag_Status information,searchable_address
1,786,F1103R_HRA38448,currently registered,de,Apotheke in Nikolassee e.K.,None,2018-11-09T18:29:04Z,1,1,1,...,Berlin,Berlin (Charlottenburg),HRA,38448,None,None,None,None,None,NaN
3,1453,M1201_HRA49032,currently registered,de,Westend Apotheke Homira Alavi e.K.,"Brentano Straße 29, 60325 Frankfurt am Main.",2018-07-25T13:36:39Z,1,1,1,...,Frankfurt am Main,Frankfurt am Main,HRA,49032,None,None,None,None,None,"Brentano Straße 29, 60325 Frankfurt am Main. F..."
5,2541,R1402_HRA5334,currently registered,de,"Bären-Apotheke, Inh.: Olaf Orthen e.K.",None,2018-07-25T14:18:03Z,1,1,1,...,Kempen,Krefeld,HRA,5334,None,None,None,None,None,NaN
6,2817,B8535_HRA440044,currently registered,de,"Apotheke Billigheim Wolfgang Pretzsch-Eckhard,...",None,2018-11-09T09:59:11Z,1,1,1,...,Billigheim,Mannheim,HRA,440044,None,None,None,None,None,NaN
7,2989,B8535_HRA340724,currently registered,de,Stadt-Apotheke Volker Blum e.K.,None,2018-11-09T09:58:28Z,1,1,1,...,Sinsheim,Mannheim,HRA,340724,None,None,None,None,None,NaN


In [30]:
# Get geometry for registry data (https://geopy.readthedocs.io/en/stable/)
geolocator = Nominatim(user_agent="example app")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=3)
registry_df["loc"] = registry_df["searchable_address"].apply(geolocator.geocode)
registry_df["point"]= registry_df["loc"].apply(lambda loc: tuple(loc.point) if loc else None)
registry_df[['lat', 'lon', 'altitude']] = pd.DataFrame(registry_df['point'].to_list(), index=df.index)
registry_df

GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Str.+Oppach&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))

In [ ]:
conn.close()

### Playground

In [ ]:
# Error catching - no all addresses are conclusive
def augment(x):
    try:
        return x.geolocator.geocode()
    except:
        return 'error:' + str(x)

test["loc"] = test["registered_address"].apply(lambda x: augment(x))

In [ ]:
test['registered_address'].str.contains('Hauptstr.')